## A script to clean, organize, and convert model output data into data frames for future visualisation

In [1]:
import pandas as pd
import numpy as np
import re
import io

In [2]:
#defining list of IDed benes for easy access later

mort_benes = [100701699, 100701792]
hosp_benes = [133365053, 133367598]

### getting demographic info

In [154]:
demo_df = pd.read_csv('/path.csv',
          usecols = ['DESY_SORT_KEY','COUNTY_CODE','STATE_CODE','ESRD_INDICATOR','AGE','RACE_CODE','SEX_CODE'])
demo_df.head()

,DESY_SORT_KEY,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ESRD_INDICATOR
0,100000019,7,50,2,4,78,0
1,100000379,45,970,2,1,78,0
2,100000421,33,700,2,1,44,0
3,100000533,33,331,1,2,78,0
4,100000711,10,340,2,1,78,0


In [97]:
#getting demo info for IDed mort benes plus 20 extra

mort_demo_info = demo_df.loc[demo_df['DESY_SORT_KEY'] == mort_benes[0]]
county_code = mort_demo_info['COUNTY_CODE'].iloc[0]
df = demo_df.query('COUNTY_CODE == @county_code')
df = df[:20]
mort_demo_info = pd.concat([mort_demo_info,df], axis=0)
mort_demo_info.reset_index(drop=True, inplace=True)
mort_demo_info.head()

,DESY_SORT_KEY,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ESRD_INDICATOR
0,100701699,45,911,1,1,77,0
1,100701699,45,911,1,1,77,0
2,100724719,45,911,1,1,78,0
3,102011037,26,911,2,1,51,0
4,102111493,45,911,2,1,78,0


In [77]:
#getting demo info for IDed hosp benes plus 20 extra

hosp_demo_info = demo_df.loc[demo_df['DESY_SORT_KEY'] == hosp_benes[0]] #only need this line after getting 20 benes
county_code = hosp_demo_info['COUNTY_CODE'].iloc[0]  
df = demo_df.query('COUNTY_CODE == @county_code')
df = df[:20] #where the comment out begins if manual process
hosp_demo_info = pd.concat([hosp_demo_info,df], axis=0)
hosp_demo_info.reset_index(drop=True, inplace=True)
hosp_demo_info.head()

In [78]:
#manual process to do if demo benes do not exist in scr_df - have to look to see if each has a score 
#error that shows this is ValueError: Columns must be same length as key
#must comment out code block above before running

# verified_list = [499878031, 499885031, 499890233, 499899763, 499923709, 499932243, 499936173, 499943251, 499946231, 499950831,
# 499671337, 499677107, 499679529, 499744659, 499750853, 499809971, 499815631, 499591921, 499624639, 499633139]

# related_bene_list = []

# for i in verified_list:
#     related_bene = demo_df.loc[demo_df['DESY_SORT_KEY'] == i]
#     related_bene_list.append(related_bene)
    
# hosp_demo_info = hosp_demo_info.append(related_bene_list)
# hosp_demo_info.reset_index(drop=True, inplace=True)
# hosp_demo_info.drop_duplicates(inplace=True)
# hosp_demo_info.shape    #should be (21,7)

(21, 7)

### getting model outputs

In [155]:
scr_df = pd.read_csv('path.csv.gz')
scr_df.rename(columns = {'XGB_SCR':'risk_mort'}, inplace = True)
scr_df.head()

,DESY_SORT_KEY,risk_mort,TAG_STANDARD,MI_VAR_0,MI_VAR_1,MI_VAR_2,MI_VAR_3,MI_VAR_4,MI_VAR_5,MI_VAR_6,...,MI_VAR_55,MI_VAR_56,MI_VAR_57,MI_VAR_58,MI_VAR_59,MI_VAR_60,MI_VAR_61,MI_VAR_62,MI_VAR_63,MI_VAR_65
0,100000019,0.068987,,HI_WOE_CAT_D_ALL:0.43638834,HI_CH__3MPQX_DC_2:0.39438298,HI_MCE_SCR:0.27611825,HI_WOE_CAT_H_ALL:0.25145298,HI_CH__0_1M_DC_2:0.23329416,HI_RNN_SCR:0.11443496,HI_TOT_0_1M_CLM_AMT:0.08204598,...,HI_WOE_CAT_AGE:-0.052813686,HI_TOT_CLM_CNT:-0.052830186,HI_CH__3MPQX_DC_13:-0.059503384,HI_DAYS_SINCE_CLM_O:-0.060049746,HI_TOT_CLM_AMT:-0.073450275,HI_C_3MPQX_CLM_CNT:-0.08313456,HI_DAYS_SINCE_CLM_C:-0.08445835,HI_WOE_CAT_RACE_CODE:-0.12992072,HI_WOE_CAT_D_H1_VR_UNQ:-0.14251709,HI_WOE_CAT_SEX_CODE : -0.20056963
1,100000379,0.098768,,HI_RNN_SCR:0.6581655,HI_WOE_CAT_D_ALL:0.41766033,HI_WOE_CAT_H_ALL:0.25576523,HI_MCE_SCR:0.15301658,HI_CH__3MPQX_DC_2:0.114177555,HI_WOE_CAT_D_H1_VR_UNQ:0.10570937,HI_DAYS_SINCE_CLM_I:0.058406986,...,HI_CH__0_1M_HC_Q:-0.022634927,HI_CH__3MPQX_DC_13:-0.025823297,HI_TOT_CLM_AMT:-0.027700365,HI_TOT_CLM_CNT:-0.037139963,HI_C_3MPQX_CLM_CNT:-0.03978723,HI_CH__3MPQX_DC_7:-0.04665029,HI_WOE_CAT_ESRD_INDICATOR:-0.055024914,HI_WOE_CAT_AGE:-0.07054309,HI_C_3MPQX_DSC_NPI:-0.095398605,HI_WOE_CAT_SEX_CODE : -0.23354445
2,100000421,0.020605,,HI_WOE_CAT_D_ALL:0.3278926,HI_CH__3MPQX_DC_5:0.30477017,HI_WOE_CAT_D_H1_VR_UNQ:0.14544646,HI_WOE_CAT_H_ALL:0.13007826,HI_WOE_CAT_H_H1_VR_UNQ:0.07358651,HI_TOT_DAYS_STAY:0.073558494,HI_TOT_3MPQX_DAYS_STAY:0.04560297,...,HI_TOT_3MPQX_CLM_AMT:-0.035050113,HI_C_3MPQX_DSC_NPI:-0.041795824,HI_CH__3MPQX_DC_2:-0.042376917,HI_DAYS_SINCE_CLM_O:-0.043420862,HI_TOT_CLM_AMT:-0.08456263,HI_WOE_CAT_ESRD_INDICATOR:-0.092920765,HI_WOE_CAT_AGE:-0.20846343,HI_RNN_SCR:-0.2636041,HI_WOE_CAT_SEX_CODE:-0.34601367,HI_MCE_SCR : -0.37909594
3,100000533,0.021137,,HI_DAYS_SINCE_CLM_HR:0.25948757,HI_WOE_CAT_SEX_CODE:0.14194864,HI_TOT_CLM_CNT:0.13056949,HI_MCE_SCR:0.1280577,HI_WOE_CAT_AGE:0.08250646,HI_DAYS_SINCE_CLM_HC:0.07667814,HI_TOT_CLM_AMT:0.06929327,...,HI_CH__3MPQX_DC_5:-0.045246433,HI_DAYS_SINCE_CLM_I:-0.049472902,HI_WOE_CAT_STATE_CODE:-0.05074788,HI_WOE_CAT_RACE_CODE:-0.059301578,HI_DAYS_SINCE_CLM_O:-0.0766719,HI_WOE_CAT_ESRD_INDICATOR:-0.08548462,HI_WOE_CAT_COUNTY_CODE:-0.10086728,HI_WOE_CAT_D_H1_VR_UNQ:-0.21273625,HI_RNN_SCR:-0.36046168,HI_WOE_CAT_D_ALL : -0.43251798
4,100000711,0.039531,,HI_WOE_CAT_D_ALL:0.3800746,HI_WOE_CAT_H_ALL:0.2704491,HI_MCE_SCR:0.24768394,HI_RNN_SCR:0.24220929,HI_WOE_CAT_D_H1_VR_UNQ:0.14432415,HI_CH__3MPQX_DC_13:0.08323514,HI_TOT_3MPQX_DAYS_STAY:0.041519836,...,HI_TOT_CLM_AMT:-0.04572407,HI_TOT_3MPQX_CLM_AMT:-0.047894776,HI_C_3MPQX_AMT:-0.055162158,HI_DAYS_SINCE_CLM_I:-0.060853798,HI_DAYS_SINCE_CLM_O:-0.06294508,HI_WOE_CAT_ESRD_INDICATOR:-0.074512206,HI_WOE_CAT_AGE:-0.08919072,HI_C_3MPQX_DSC_NPI:-0.097561985,HI_DAYS_SINCE_CLM_C:-0.10481243,HI_WOE_CAT_SEX_CODE : -0.23446992


In [7]:
#defining fn to get the top xgb model ouputs names and values per bene

def get_xgb_outputs(bene):
    df = scr_df.loc[scr_df['DESY_SORT_KEY'] == bene]
    col_list = list(df.columns[3:])
    head_df = df[df.columns[:3]]
    tail_df = df[col_list]
    
    for col in col_list:
        head_df[[col, col+'_val']] = tail_df[col].str.split(":", expand=True)
    
    df = head_df
    
    return df

In [167]:
#getting values for IDed mort benes

mort_xgb_outputs = get_xgb_outputs(mort_benes[0])
mort_xgb_outputs.head()

,DESY_SORT_KEY,risk_mort,TAG_STANDARD,MI_VAR_0,MI_VAR_0_val,MI_VAR_1,MI_VAR_1_val,MI_VAR_2,MI_VAR_2_val,MI_VAR_3,...,MI_VAR_60,MI_VAR_60_val,MI_VAR_61,MI_VAR_61_val,MI_VAR_62,MI_VAR_62_val,MI_VAR_63,MI_VAR_63_val,MI_VAR_65,MI_VAR_65_val
4960,100701699,0.571055,,HI_MCE_SCR,1.5005966,HI_RNN_SCR,1.2949206,HI_WOE_CAT_D_ALL,0.43584278,HI_WOE_CAT_D_H1_VR_UNQ,...,HI_CH__3MPQX_DC_7,-0.041067936,HI_DAYS_SINCE_CLM_O,-0.042412136,HI_DAYS_SINCE_CLM_I,-0.045245543,HI_CH__3MPQX_DC_2,-0.050646015,HI_TOT_CLM_AMT,-0.091541365


In [100]:
#getting TAG_STANDARD value

mort_features = pd.read_csv('path.csv.gz')
mort_features = mort_features[mort_features['DESY_SORT_KEY'] == mort_benes[0]]
mort_features['TAG_STANDARD']

4960    0
Name: TAG_STANDARD, dtype: int64

In [101]:
mort_xgb_outputs['TAG_STANDARD'] = 0
mort_xgb_outputs.head()

,DESY_SORT_KEY,risk_mort,TAG_STANDARD,MI_VAR_0,MI_VAR_0_val,MI_VAR_1,MI_VAR_1_val,MI_VAR_2,MI_VAR_2_val,MI_VAR_3,...,MI_VAR_60,MI_VAR_60_val,MI_VAR_61,MI_VAR_61_val,MI_VAR_62,MI_VAR_62_val,MI_VAR_63,MI_VAR_63_val,MI_VAR_65,MI_VAR_65_val
4960,100701699,0.571055,0,HI_MCE_SCR,1.5005966,HI_RNN_SCR,1.2949206,HI_WOE_CAT_D_ALL,0.43584278,HI_WOE_CAT_D_H1_VR_UNQ,...,HI_CH__3MPQX_DC_7,-0.041067936,HI_DAYS_SINCE_CLM_O,-0.042412136,HI_DAYS_SINCE_CLM_I,-0.045245543,HI_CH__3MPQX_DC_2,-0.050646015,HI_TOT_CLM_AMT,-0.091541365


In [102]:
#getting the the xgb scores for all the related mort benes

df = mort_demo_info[1:21]
related_bene_list = []

for bene in list(df['DESY_SORT_KEY']):
    related_bene = get_xgb_outputs(bene)
    related_bene_list.append(related_bene)
    
mort_xgb_outputs = mort_xgb_outputs.append(related_bene_list)
mort_xgb_outputs.reset_index(drop=True, inplace=True)
mort_xgb_outputs.shape

(21, 133)

In [12]:
scr_df = pd.read_csv('path.csv.gz')
scr_df.rename(columns = {'XGB_SCR':'risk_hosp'}, inplace = True)
scr_df.head()

,DESY_SORT_KEY,risk_hosp,TAG_OPEN,MI_VAR_0,MI_VAR_1,MI_VAR_2,MI_VAR_3,MI_VAR_4,MI_VAR_5,MI_VAR_6,...,MI_VAR_55,MI_VAR_56,MI_VAR_57,MI_VAR_58,MI_VAR_59,MI_VAR_60,MI_VAR_61,MI_VAR_62,MI_VAR_63,MI_VAR_65
0,100000379,0.008122,,HI_DAYS_SINCE_CLM_C:0.34583282,HI_CH__0_1M_DC_9:0.1538564,HI_TOT__0_1M__NPI:0.14798091,HI_WOE_CAT_D_ALL:0.10876898,HI_TOT_0_1M_CLM_AMT:0.04641707,HI_V_3MPQX_CLM_CNT:0.0378675,HI_TOT_0_1M_CLM_CNT:0.037020937,...,HI_I_3MPQX_CLM_CNT:-0.049679812,HI_C_0_1M_AMT:-0.061778177,HI_TOT_3MPQX_CLM_CNT:-0.063469306,HI_MCE_SCR:-0.06559053,HI_TOT__3MPQX__NPI:-0.06798861,HI_C_3MPQX_CLM_CNT:-0.118729524,HI_DAYS_SINCE_LAST_ACSC:-0.16027461,HI_PRIOR_UNPLN_ADM:-0.21087384,HI_RNN_SCR:-0.28997225,HI_NUM_ACSC : -0.3130976
1,100000421,0.015072,,HI_DAYS_SINCE_CLM_C:0.43069646,HI_TOT__0_1M__NPI:0.34275162,HI_C_0_1M_DSC_NPI:0.2827577,HI_WOE_CAT_D_H1_VR_UNQ:0.2261787,HI_TOT_0_1M_CLM_AMT:0.11835694,HI_TOT_0_1M_CLM_CNT:0.08002413,HI_DAYS_SINCE_CLM_O:0.07302347,...,HI_I_3MPQX_CLM_CNT:-0.0398046,HI_WOE_CAT_COUNTY_CODE:-0.048345227,HI_CH__3MPQX_DC_11:-0.056556594,HI_TOT__3MPQX__NPI:-0.0684173,HI_C_3MPQX_CLM_CNT:-0.09045306,HI_TOT_3MPQX_CLM_CNT:-0.10414194,HI_NUM_ACSC:-0.23801601,HI_RNN_SCR:-0.23859762,HI_PRIOR_UNPLN_ADM:-0.3183128,HI_MCE_SCR : -0.5629073
2,100000711,0.004444,,HI_PRIOR_UNPLN_ADM:0.21221393,HI_WOE_CAT_D_H1_VR_UNQ:0.1633006,HI_I_3MPQX_DSC_NPI:0.055686746,HI_I_3MPQX_CLM_CNT:0.031987816,HI_DAYS_SINCE_CLM_I:0.028845737,HI_WOE_CAT_D_ALL:0.025259597,HI_I_3MPQX_AMT:0.023735544,...,HI_TOT__3MPQX__NPI:-0.04531999,HI_TOT__0_1M__NPI:-0.04629512,HI_C_3MPQX_AMT:-0.05337566,HI_TOT_3MPQX_CLM_CNT:-0.055126,HI_C_0_1M_AMT:-0.068237744,HI_C_3MPQX_CLM_CNT:-0.08868351,HI_RNN_SCR:-0.09089945,HI_DAYS_SINCE_CLM_C:-0.116515644,HI_NUM_ACSC:-0.26164195,HI_DAYS_SINCE_LAST_ACSC : -0.8186775
3,100001101,0.003485,,HI_WOE_CAT_D_ALL:0.10864182,HI_WOE_CAT_D_H1_VR_UNQ:0.09167408,HI_DAYS_SINCE_CLM_O:0.08617897,HI_CH__0_1M_DC_18:0.04265873,HI_WOE_CAT_RACE_CODE:0.016146714,HI_O_0_1M_AMT:0.01476056,HI_WOE_CAT_COUNTY_CODE:0.013594808,...,HI_DAYS_SINCE_CLM_I:-0.041652888,HI_CH__0_1M_DC_9:-0.042518027,HI_CH__3MPQX_DC_11:-0.059630662,HI_C_3MPQX_CLM_CNT:-0.07304262,HI_RNN_SCR:-0.09359014,HI_DAYS_SINCE_CLM_C:-0.1103653,HI_MCE_SCR:-0.1755477,HI_PRIOR_UNPLN_ADM:-0.21578638,HI_NUM_ACSC:-0.310469,HI_DAYS_SINCE_LAST_ACSC : -0.56167185
4,100001395,0.001206,,HI_TOT_3MPQX_CLM_CNT:0.10888413,HI_C_3MPQX_AMT:0.047452565,HI_CH__3MPQX_DC_11:0.047376122,HI_TOT__3MPQX__NPI:0.045667388,HI_TOT_0_1M_CLM_AMT:0.03967768,HI_WOE_CAT_STATE_CODE:0.02381135,HI_WOE_CAT_RACE_CODE:0.013941393,...,HI_WOE_CAT_D_ALL:-0.05350364,HI_DAYS_SINCE_CLM_I:-0.05658709,HI_TOT__0_1M__NPI:-0.06414834,HI_DAYS_SINCE_CLM_C:-0.091970004,HI_PRIOR_UNPLN_ADM:-0.22062361,HI_RNN_SCR:-0.2207863,HI_NUM_ACSC:-0.31887951,HI_WOE_CAT_D_H1_VR_UNQ:-0.3252271,HI_DAYS_SINCE_LAST_ACSC:-0.61217725,HI_MCE_SCR : -0.6654002


In [13]:
#getting values for IDed hosp benes

hosp_xgb_outputs = get_xgb_outputs(hosp_benes[0])
hosp_xgb_outputs.head()

,DESY_SORT_KEY,risk_hosp,TAG_OPEN,MI_VAR_0,MI_VAR_0_val,MI_VAR_1,MI_VAR_1_val,MI_VAR_2,MI_VAR_2_val,MI_VAR_3,...,MI_VAR_60,MI_VAR_60_val,MI_VAR_61,MI_VAR_61_val,MI_VAR_62,MI_VAR_62_val,MI_VAR_63,MI_VAR_63_val,MI_VAR_65,MI_VAR_65_val
37102,133365053,0.088263,,HI_DAYS_SINCE_CLM_C,0.42237523,HI_MCE_SCR,0.4112783,HI_WOE_CAT_D_H1_VR_UNQ,0.3510618,HI_DAYS_SINCE_LAST_ACSC,...,HI_C_3MPQX_CLM_CNT,-0.0522184,HI_GD_C_0_2M_AMT,-0.109436475,HI_RNN_SCR,-0.12201724,HI_C_0_1M_AMT,-0.18462016,HI_PRIOR_UNPLN_ADM,-0.21640384


In [14]:
#getting TAG_OPEN value

hosp_features = pd.read_csv('path.csv.gz')
hosp_features = hosp_features[hosp_features['DESY_SORT_KEY'] == hosp_benes[0]]
hosp_features['TAG_OPEN']

37102    0
Name: TAG_OPEN, dtype: int64

In [85]:
hosp_xgb_outputs['TAG_OPEN'] = 0
hosp_xgb_outputs.head()

,DESY_SORT_KEY,risk_hosp,TAG_OPEN,MI_VAR_0,MI_VAR_0_val,MI_VAR_1,MI_VAR_1_val,MI_VAR_2,MI_VAR_2_val,MI_VAR_3,...,MI_VAR_60,MI_VAR_60_val,MI_VAR_61,MI_VAR_61_val,MI_VAR_62,MI_VAR_62_val,MI_VAR_63,MI_VAR_63_val,MI_VAR_65,MI_VAR_65_val
37102,133365053,0.088263,0,HI_DAYS_SINCE_CLM_C,0.42237523,HI_MCE_SCR,0.4112783,HI_WOE_CAT_D_H1_VR_UNQ,0.3510618,HI_DAYS_SINCE_LAST_ACSC,...,HI_C_3MPQX_CLM_CNT,-0.0522184,HI_GD_C_0_2M_AMT,-0.109436475,HI_RNN_SCR,-0.12201724,HI_C_0_1M_AMT,-0.18462016,HI_PRIOR_UNPLN_ADM,-0.21640384


In [1]:
#getting the the xgb scores for all the related hosp benes

df = hosp_demo_info[1:21]
related_bene_list = []

for bene in list(df['DESY_SORT_KEY']):
    related_bene = get_xgb_outputs(bene)
    related_bene_list.append(related_bene)
    
hosp_xgb_outputs = hosp_xgb_outputs.append(related_bene_list)
hosp_xgb_outputs.reset_index(drop=True, inplace=True)
hosp_xgb_outputs.shape

In [105]:
mort_df = pd.merge(mort_demo_info, mort_xgb_outputs, how='left', on='DESY_SORT_KEY')
mort_df.drop_duplicates(inplace=True)

In [106]:
mort_df.shape

(21, 139)

In [90]:
hosp_df = pd.merge(hosp_demo_info, hosp_xgb_outputs, how='left', on='DESY_SORT_KEY')

In [91]:
hosp_df.shape

(21, 139)

## rnn scores

In [2]:
rnn_scores_mort = open(r"/standard_rnn_reason_codes", "r").read()

In [3]:
rnn_attribution_mort = pd.read_fwf(io.StringIO(rnn_scores_mort), skiprows=1, header=None, index_col=False,
                                   names=['col1'], widths=[2000])

In [109]:
rnn_attribution_mort.head()

,col1
0,"100701699,0.37251,111,-0.01773,['DMII wo cmp n..."
1,"100701699,0.37251,114,-0.01576,['Malignant hyp..."
2,"100701699,0.37251,117,-0.00917,['DMII oth nt s..."
3,"100701699,0.37251,110,-0.00646,['DMII wo cmp n..."
4,"100701699,0.37251,112,-0.00543,['DMII oth nt s..."


In [110]:
rnn_attribution_mort[['DESY_SORT_KEY','RNN_score','visit_number', 'visit_attribution','diagnosis']] = rnn_attribution_mort['col1'].str.split(',',n=4, expand=True)
rnn_attribution_mort.drop(['col1', 'RNN_score'], axis=1, inplace=True)
rnn_attribution_mort['DESY_SORT_KEY'] = rnn_attribution_mort['DESY_SORT_KEY'].astype(int)
rnn_attribution_mort.head()

,DESY_SORT_KEY,visit_number,visit_attribution,diagnosis
0,100701699,111,-0.01773,"['DMII wo cmp nt st uncntr', 'Malignant hypert..."
1,100701699,114,-0.01576,"['Malignant hypertension', 'Prim cardiomyopath..."
2,100701699,117,-0.00917,"['DMII oth nt st uncntrld', 'Toxic encephalopa..."
3,100701699,110,-0.00646,"['DMII wo cmp nt st uncntr', 'Dehydration', 'M..."
4,100701699,112,-0.00543,"['DMII oth nt st uncntrld', 'Malignant hyperte..."


In [111]:
#manipulating the df so that it is in the correct format

mort1_rnn = rnn_attribution_mort
mort1_rnn.sort_values('visit_attribution', ascending=False, inplace=True)

visit_attribution = pd.DataFrame(mort1_rnn['visit_attribution'])
diagnosis = pd.DataFrame(mort1_rnn['diagnosis'])

visit_attribution = visit_attribution.T
diagnosis = diagnosis.T

visit_attribution.reset_index(drop=True, inplace = True)
diagnosis.reset_index(drop=True, inplace = True)

visit_attribution_cols = ['visit_high1val', 'visit_high2val', 'visit_high3val', 'visit_high4val', 'visit_high5val',
            'visit_low1val', 'visit_low2val', 'visit_low3val', 'visit_low4val', 'visit_low5val']

diagnosis_cols = ['diagnosis_high1', 'diagnosis_high2', 'diagnosis_high3', 'diagnosis_high4', 'diagnosis_high5',
            'diagnosis_low1', 'diagnosis_low2', 'diagnosis_low3', 'diagnosis_low4', 'diagnosis_low5']

visit_attribution.columns = visit_attribution_cols
diagnosis.columns = diagnosis_cols

diagnosis['DESY_SORT_KEY'] = mort_benes[0]

In [112]:
#putting data back together
mort1_rnn = pd.concat([visit_attribution, diagnosis], axis=1)

In [113]:
mort_df = pd.merge(mort_df, mort1_rnn, how='left', on='DESY_SORT_KEY')
mort_df.shape

(21, 159)

In [114]:
rnn_scores_hosp = open(r"/open_rnn_reason_codes", "r").read()
rnn_attribution_hosp = pd.read_fwf(io.StringIO(rnn_scores_hosp), skiprows=1, header=None, index_col=False,
                                   names=['col1'], widths=[2000])
rnn_attribution_hosp.head()

,col1
0,"133365053,0.00312,101,-0.00114,['Malignant hyp..."
1,"133365053,0.00312,105,-0.00102,['Anemia NOS', ..."
2,"133365053,0.00312,102,-0.00057,['Malignant hyp..."
3,"133365053,0.00312,83,-0.00044,['Malignant hype..."
4,"133365053,0.00312,103,-0.00044,['Cereb degener..."


In [115]:
rnn_attribution_hosp[['DESY_SORT_KEY','RNN_score','visit_number', 'visit_attribution','diagnosis']] = rnn_attribution_hosp['col1'].str.split(',',n=4, expand=True)
rnn_attribution_hosp.drop(['col1', 'RNN_score'], axis=1, inplace=True)
rnn_attribution_hosp['DESY_SORT_KEY'] = rnn_attribution_hosp['DESY_SORT_KEY'].astype(int)
rnn_attribution_hosp.head()

,DESY_SORT_KEY,visit_number,visit_attribution,diagnosis
0,133365053,101,-0.00114,"['Malignant hypertension', 'Hypertension NOS',..."
1,133365053,105,-0.00102,"['Anemia NOS', 'Malignant hypertension', 'Late..."
2,133365053,102,-0.00057,"['Malignant hypertension', 'Chr airway obstruc..."
3,133365053,83,-0.00044,"['Malignant hypertension', 'Hypertension NOS',..."
4,133365053,103,-0.00044,"['Cereb degeneration NOS', 'Malignant hyperten..."


In [116]:
hosp1_rnn = rnn_attribution_hosp
hosp1_rnn.sort_values('visit_attribution', ascending=False, inplace=True)

visit_attribution = pd.DataFrame(hosp1_rnn['visit_attribution'])
diagnosis = pd.DataFrame(hosp1_rnn['diagnosis'])

visit_attribution = visit_attribution.T
diagnosis = diagnosis.T

visit_attribution.reset_index(drop=True, inplace = True)
diagnosis.reset_index(drop=True, inplace = True)

visit_attribution.columns = visit_attribution_cols
diagnosis.columns = diagnosis_cols

diagnosis['DESY_SORT_KEY'] = hosp_benes[0]

In [117]:
hosp1_rnn = pd.concat([visit_attribution, diagnosis], axis=1)
hosp_df = pd.merge(hosp_df, hosp1_rnn, how='left', on='DESY_SORT_KEY')
hosp_df.shape

(21, 159)

## mce scores

In [2]:
mce_mort = open(r"/mort_MCE_reason_codes.txt", "r").read()

In [119]:
bene_info = re.split(r"Bene", mce_mort)
bene_info.pop(0) #getting rid of the empty string at index 0

'\n \n '

In [12]:
#code to create the first iteration of the mce output data. comment out if using other method

# bene_info[0]

# desy_pattern = r'10000\d\d\d\d'
# scr_pattern = r'=(.*\d\d)'
# label_pattern = 'label\s(\d)'

# bene_list = []

# for bene in bene_info:
#     bene_dict = {}
    
#     #find all DESY KEYS that are in the 10000 group
#     DESY = re.search(desy_pattern, bene)
#     if DESY is not None:
#         DESY_SORT_KEY = DESY.group()
#     else:
#         continue
        
#     scr = re.search(scr_pattern, bene)
#     mort_scr = scr.group(1)
    
#     label = re.search(label_pattern, bene)
#     mort_label = label.group(1)
    
#     bene_dict['DESY_SORT_KEY'] = DESY_SORT_KEY
#     bene_dict['mort_scr'] = mort_scr
#     bene_dict['mort_label'] = mort_label
    
#     bene_list.append(bene_dict)

In [193]:
#first iteration of mce ouput data - continued

# bene_list

# bene_df = pd.DataFrame(bene_list)
# bene_df['DESY_SORT_KEY'] = bene_df['DESY_SORT_KEY'].astype(int)

# bene_df.head()

In [120]:
#second iteration of mce output data for mort benes

ICD_df = pd.read_fwf(io.StringIO(bene_info[0]), skiprows=1, header=None, index_col=False,
                     names=['col_1'], widths=[2000])
ICD_df.head()

,col_1
0,0.001901 20150925 4280 CHF NOS
1,0.001492 20150831 4019 Hypertension NOS
2,0.001466 20150420 2859 Anemia NOS
3,0.001438 20150512 72887 Muscle weakness-general
4,0.001422 20150422 78605 Shortness of breath


In [121]:
ICD_df[['ICD_code_importance','date','ICD_code','ICD_code_desc']] = ICD_df['col_1'].str.split(n=3, expand=True)
ICD_df.drop(['col_1'], axis=1, inplace=True)
ICD_df.drop(['ICD_code'], axis=1, inplace=True)
ICD_df.drop(['date'], axis=1, inplace=True)
ICD_df['ICD_code_importance'] = ICD_df['ICD_code_importance'].astype(float)

#getting top 5 and low 5 values
ICD_df.sort_values('ICD_code_importance', ascending=False, inplace=True)
df1 = ICD_df.iloc[:5]
df2 = ICD_df.iloc[-5:]
ICD_df = df1.append(df2)
ICD_df.reset_index(drop=True, inplace=True)


#manipulating the df so that it is in the correct format
ICD_code_desc = pd.DataFrame(ICD_df['ICD_code_desc'])
ICD_code_importance = pd.DataFrame(ICD_df['ICD_code_importance'])

ICD_code_desc = ICD_code_desc.T
ICD_code_importance = ICD_code_importance.T

ICD_code_desc.reset_index(drop=True, inplace = True)
ICD_code_importance.reset_index(drop=True, inplace = True)

desc_cols = ['icd_code_high1', 'icd_code_high2', 'icd_code_high3', 'icd_code_high4', 'icd_code_high5',
            'icd_code_low1', 'icd_code_low2', 'icd_code_low3', 'icd_code_low4', 'icd_code_low5']

val_cols = ['icd_code_high1val', 'icd_code_high2val', 'icd_code_high3val', 'icd_code_high4val', 'icd_code_high5val',
            'icd_code_low1val', 'icd_code_low2val', 'icd_code_low3val', 'icd_code_low4val', 'icd_code_low5val']

ICD_code_desc.columns = desc_cols
ICD_code_importance.columns = val_cols

ICD_code_importance['DESY_SORT_KEY'] = mort_benes[0]

In [122]:
ICD_code_importance.head()

,icd_code_high1val,icd_code_high2val,icd_code_high3val,icd_code_high4val,icd_code_high5val,icd_code_low1val,icd_code_low2val,icd_code_low3val,icd_code_low4val,icd_code_low5val,DESY_SORT_KEY
0,0.001901,0.001492,0.001466,0.001438,0.001422,-0.000375,-0.000383,-0.000386,-0.000386,-0.000386,100701699


In [123]:
#putting data back together
mort_icd = pd.concat([ICD_code_desc, ICD_code_importance], axis=1)

In [124]:
mort_df = pd.merge(mort_df, mort_icd, how='left', on='DESY_SORT_KEY')
mort_df.shape

(21, 179)

In [3]:
# mce model output for hosp benes

mce_hosp = open(r"/open_MCE_reason_codes.txt", "r").read()
bene_info = re.split(r"Bene", mce_hosp)
bene_info.pop(0) #getting rid of the empty string at index 0

ICD_df = pd.read_fwf(io.StringIO(bene_info[0]), skiprows=1, header=None, index_col=False,
                     names=['col_1'], widths=[2000])
ICD_df.head()

,col_1
0,0.003229 20150923 4019 Hypertension NOS
1,0.002608 20150520 78605 Shortness of breath
2,0.002576 20150923 4280 CHF NOS
3,0.002256 20150922 496 Chr airway obstruct NEC
4,0.002083 20150523 40390 Hy kid NOS w cr kid I-IV


In [126]:
ICD_df[['ICD_code_importance','date','ICD_code','ICD_code_desc']] = ICD_df['col_1'].str.split(n=3, expand=True)
ICD_df.drop(['col_1'], axis=1, inplace=True)
ICD_df.drop(['ICD_code'], axis=1, inplace=True)
ICD_df.drop(['date'], axis=1, inplace=True)
ICD_df['ICD_code_importance'] = ICD_df['ICD_code_importance'].astype(float)

#getting top 5 and low 5 values
ICD_df.sort_values('ICD_code_importance', ascending=False, inplace=True)
df1 = ICD_df.iloc[:5]
df2 = ICD_df.iloc[-5:]
ICD_df = df1.append(df2)
ICD_df.reset_index(drop=True, inplace=True)


#manipulating the df so that it is in the correct format
ICD_code_desc = pd.DataFrame(ICD_df['ICD_code_desc'])
ICD_code_importance = pd.DataFrame(ICD_df['ICD_code_importance'])

ICD_code_desc = ICD_code_desc.T
ICD_code_importance = ICD_code_importance.T

ICD_code_desc.reset_index(drop=True, inplace = True)
ICD_code_importance.reset_index(drop=True, inplace = True)

ICD_code_desc.columns = desc_cols
ICD_code_importance.columns = val_cols

ICD_code_importance['DESY_SORT_KEY'] = hosp_benes[0]

In [127]:
ICD_code_importance.head()

,icd_code_high1val,icd_code_high2val,icd_code_high3val,icd_code_high4val,icd_code_high5val,icd_code_low1val,icd_code_low2val,icd_code_low3val,icd_code_low4val,icd_code_low5val,DESY_SORT_KEY
0,0.003229,0.002608,0.002576,0.002256,0.002083,-0.000094,-0.000098,-0.000183,-0.000206,-0.000224,133365053


In [128]:
hosp_icd = pd.concat([ICD_code_desc, ICD_code_importance], axis=1)
hosp_df = pd.merge(hosp_df, hosp_icd, how='left', on='DESY_SORT_KEY')
hosp_df.shape

(21, 179)

## ccw mapping

In [129]:
mce_mort = open(r"/mort_MCE_reason_codes.txt", "r").read()
bene_info = re.split(r"Bene", mce_mort)
bene_info.pop(0) #getting rid of the empty string at index 0

ICD_mort_df = pd.read_fwf(io.StringIO(bene_info[0]), skiprows=1, header=None, index_col=False,
                     names=['col_1'], widths=[2000])
ICD_mort_df[['ICD_code_importance','date','ICD_code','ICD_code_desc']] = ICD_mort_df['col_1'].str.split(n=3, expand=True)
icd_code_formatted_mort = ICD_mort_df['ICD_code'].str[:3] + '.' + ICD_mort_df['ICD_code'].str[3:]


In [130]:
ccw_map = pd.read_csv('ccw_map.csv')
icd9 = list(ccw_map['icd9'])

In [131]:
cc_list = []

for icd_code in icd_code_formatted_mort:
    for index in range(len(icd9)):
        if icd_code in icd9[index]:
            chronic_condition = ccw_map['chronic_condition'][index]
            cc_map = icd_code+', '+chronic_condition
            print('Condition: {}, ICD Code: {}, Index: {}'.format(chronic_condition,icd_code,index))
        else:
           continue 
            print(icd_code)
            cc_map = [icd_code,'not available']
            
        cc_list.append(cc_map)
cc_array = np.vstack(cc_list)
df_cc_map = pd.DataFrame(cc_array, columns=['col_1'])
df_cc_map[['icd_code','chronic_condition']] = df_cc_map['col_1'].str.split(',', expand=True)
df_cc_map.drop(['col_1'], axis=1, inplace=True)

In [144]:
# CHECK FOR EXCLUSIONS using csv

df_cc_map.shape
df_cc_map.head(45)

In [136]:
num_chronic_conditions = df_cc_map['icd_code'].value_counts().to_frame()
num_chronic_conditions.reset_index(level=0, inplace=True)
num_chronic_conditions.rename(columns={'index':'icd_code', 'icd_code':'num_chronic_conditions'}, inplace=True)
num_chronic_conditions['icd_code'] = num_chronic_conditions['icd_code'].astype(str)

chronic_conditions_final = pd.merge(df_cc_map, num_chronic_conditions, how='left', on='icd_code')
chronic_conditions_final.drop_duplicates(subset=['chronic_condition'], inplace=True)

cc_mort_final = pd.DataFrame({'DESY_SORT_KEY':[mort_benes[0]],
                          'chronic_conditions':[chronic_conditions_final['chronic_condition'].to_list()],
                          'num_chronic_conditions': [chronic_conditions_final['num_chronic_conditions'].values.sum()]
                         })
cc_mort_final.head(10)

,DESY_SORT_KEY,chronic_conditions,num_chronic_conditions
0,100701699,"[ Heart Failure, Hypertension, Anemia, Chro...",11


In [139]:
mort_df_FINAL = pd.merge(mort_df, cc_mort_final, how='left', on='DESY_SORT_KEY')
mort_df_FINAL.shape

(21, 181)

In [140]:
# ccw mapping for csv

mce_hosp = open(r"/open_MCE_reason_codes.txt", "r").read()
bene_info = re.split(r"Bene", mce_hosp)
bene_info.pop(0) #getting rid of the empty string at index 0

ICD_hosp_df = pd.read_fwf(io.StringIO(bene_info[0]), skiprows=1, header=None, index_col=False,
                     names=['col_1'], widths=[2000])
ICD_hosp_df[['ICD_code_importance','date','ICD_code','ICD_code_desc']] = ICD_hosp_df['col_1'].str.split(n=3, expand=True)
icd_code_formatted_hosp = ICD_hosp_df['ICD_code'].str[:3] + '.' + ICD_hosp_df['ICD_code'].str[3:]


In [141]:
cc_list = []

for icd_code in icd_code_formatted_hosp:
    for index in range(len(icd9)):
        if icd_code in icd9[index]:
            chronic_condition = ccw_map['chronic_condition'][index]
            cc_map = icd_code+', '+chronic_condition
            print('Condition: {}, ICD Code: {}, Index: {}'.format(chronic_condition,icd_code,index))
        else:
           continue 
            print(icd_code)
            cc_map = [icd_code,'not available']
            
        cc_list.append(cc_map)
cc_array = np.vstack(cc_list)
df_cc_map = pd.DataFrame(cc_array, columns=['col_1'])
df_cc_map[['icd_code','chronic_condition']] = df_cc_map['col_1'].str.split(',', expand=True)
df_cc_map.drop(['col_1'], axis=1, inplace=True)

In [149]:
# CHECK FOR EXCLUSIONS using csv

df_cc_map.shape
df_cc_map.head(20)

In [147]:
num_chronic_conditions = df_cc_map['icd_code'].value_counts().to_frame()
num_chronic_conditions.reset_index(level=0, inplace=True)
num_chronic_conditions.rename(columns={'index':'icd_code', 'icd_code':'num_chronic_conditions'}, inplace=True)
num_chronic_conditions['icd_code'] = num_chronic_conditions['icd_code'].astype(str)

chronic_conditions_final = pd.merge(df_cc_map, num_chronic_conditions, how='left', on='icd_code')
chronic_conditions_final.drop_duplicates(subset=['chronic_condition'], inplace=True)

cc_hosp_final = pd.DataFrame({'DESY_SORT_KEY':[hosp_benes[0]],
                          'chronic_conditions':[chronic_conditions_final['chronic_condition'].to_list()],
                          'num_chronic_conditions': [chronic_conditions_final['num_chronic_conditions'].values.sum()]
                         })
cc_hosp_final.head()

,DESY_SORT_KEY,chronic_conditions,num_chronic_conditions
0,133365053,"[ Hypertension, Heart Failure, Chronic Kidne...",11


In [150]:
hosp_df_FINAL = pd.merge(hosp_df, cc_hosp_final, how='left', on='DESY_SORT_KEY')
hosp_df_FINAL.shape

(21, 181)

In [151]:
mort_df_FINAL.to_csv('mort_bene_details_final.csv', index=False)
hosp_df_FINAL.to_csv('hosp_bene_details_final.csv', index=False)

## loading in claim data

In [4]:
#loading in all col headers, claim data, and transforming data to correct format
level_5 = '13336'

#hha
hha_13_15_col_list = pd.read_csv('path.csv')
hha_13_15_col_list = list(hha_13_15_col_list['col'])
hha = pd.read_csv('/path/level_5/'+level_5+'_master_merged.csv', 
                  names=hha_13_15_col_list, header=None)
hha = hha.loc[hha['DEL_FT'] == 'A']
hha.reset_index(drop=True, inplace=True)
hha.dropna(axis=1, how='all', inplace=True)
hha['CLM_THRU_DT'] = hha['CLM_THRU_DT'].astype(str)
hha['CLM_THRU_DT'] = pd.to_datetime(hha['CLM_THRU_DT']) 


#hospice
hospice_13_15_col_list = pd.read_csv('path.csv')
hospice_13_15_col_list = list(hospice_13_15_col_list['col'])
hospice = pd.read_csv('/path/level_5/'+level_5+'_master_merged.csv', 
                  names=hospice_13_15_col_list, header=None)
hospice = hospice.loc[hospice['DEL_FT'] == 'HC'] 
hospice.reset_index(drop=True, inplace=True)
# hospice.dropna(axis=1, how='all', inplace=True)
hospice['CLM_THRU_DT'] = hospice['CLM_THRU_DT'].astype(str)
hospice['CLM_THRU_DT'] = pd.to_datetime(hospice['CLM_THRU_DT'])


#snf
snf_13_15_col_list = pd.read_csv('path.csv')
snf_13_15_col_list = list(snf_13_15_col_list['col'])
snf = pd.read_csv('/path/level_5/'+level_5+'_master_merged.csv', 
                  names=snf_13_15_col_list, header=None)
snf = snf.loc[snf['DEL_FT'] == 'S']
snf.reset_index(drop=True, inplace=True)
snf.dropna(axis=1, how='all', inplace=True)
snf['CLM_THRU_DT'] = snf['CLM_THRU_DT'].astype(str)
snf['CLM_THRU_DT'] = pd.to_datetime(snf['CLM_THRU_DT'])


#dme
dme_13_15_col_list = pd.read_csv('path.csv')
dme_13_15_col_list = list(dme_13_15_col_list['col'])
dme = pd.read_csv('/path/level_5/'+level_5+'_master_merged.csv', 
                  names=dme_13_15_col_list, header=None)
dme = dme.loc[dme['DEL_FT'] == 'V']
dme.reset_index(drop=True, inplace=True)
dme.dropna(axis=1, how='all', inplace=True)
dme['CLM_THRU_DT'] = dme['CLM_THRU_DT'].astype(str)
dme['CLM_THRU_DT'] = pd.to_datetime(dme['CLM_THRU_DT'])

#inpatient
inpatient_13_15_col_list = pd.read_csv('path.csv')
inpatient_13_15_col_list = list(inpatient_13_15_col_list['col'])
inpatient = pd.read_csv('/path/level_5/'+level_5+'_master_merged.csv', 
                  names=inpatient_13_15_col_list, header=None)
inpatient = inpatient.loc[inpatient['DEL_FT'] == 'I']
inpatient.reset_index(drop=True, inplace=True)
inpatient.dropna(axis=1, how='all', inplace=True)
inpatient['CLM_THRU_DT'] = inpatient['CLM_THRU_DT'].astype(str)
inpatient['CLM_THRU_DT'] = pd.to_datetime(inpatient['CLM_THRU_DT'])


#outpatient
outpatient_13_15_col_list = pd.read_csv('path.csv')
outpatient_13_15_col_list = list(outpatient_13_15_col_list['col'])
outpatient = pd.read_csv('/path/level_5/'+level_5+'_master_merged.csv', 
                  names=outpatient_13_15_col_list, header=None)
outpatient = outpatient.loc[outpatient['DEL_FT'] == 'O']
outpatient.reset_index(drop=True, inplace=True)
outpatient.dropna(axis=1, how='all', inplace=True)
outpatient['CLM_THRU_DT'] = outpatient['CLM_THRU_DT'].astype(str)
outpatient['CLM_THRU_DT'] = pd.to_datetime(outpatient['CLM_THRU_DT'])


#carrier
carrier_13_15_col_list = pd.read_csv('path.csv')
carrier_13_15_col_list = list(carrier_13_15_col_list['col'])
carrier = pd.read_csv('/path/level_5/'+level_5+'_master_merged.csv', 
                  names=carrier_13_15_col_list, header=None)
carrier = carrier.loc[carrier['DEL_FT'] == 'C']
carrier.reset_index(drop=True, inplace=True)
carrier.dropna(axis=1, how='all', inplace=True)
carrier['CLM_THRU_DT'] = carrier['CLM_THRU_DT'].astype(str)
carrier['CLM_THRU_DT'] = pd.to_datetime(carrier['CLM_THRU_DT'])


In [48]:
#defining variables

chosen_bene = 133365053
forecast_date = '20150930' #must be string in YYYYMMDD format

In [31]:
#fn to use to get the claims for the last 12 months

def claims_last_12m(input_df, forecast_date):
    df = input_df
    forecast_date = pd.to_datetime(forecast_date)
    lower_bound = forecast_date - pd.DateOffset(months=12)

    mask = (df['CLM_THRU_DT'] >= lower_bound) & (df['CLM_THRU_DT'] <= forecast_date)
    df = df.loc[mask]
    
    df.sort_values('CLM_THRU_DT', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [16]:
inpatient_columns = ['DESY_SORT_KEY','CLAIM_NO','ORG_NPI_NUM','CLM_ADMSN_DT','NCH_BENE_DSCHRG_DT', 'PRNCPAL_DGNS_CD']
oupatient_columns = ['DESY_SORT_KEY','CLAIM_NO','ORG_NPI_NUM','PRNCPAL_DGNS_CD','CLM_THRU_DT']
carrier_columns = ['DESY_SORT_KEY','CLAIM_NO','CLM_THRU_DT','PRNCPAL_DGNS_CD']

### mortality: inpatient, outpatient, carrier

In [17]:
inpatient = inpatient.loc[inpatient['DESY_SORT_KEY'] == mort_benes[0]]
outpatient = outpatient.loc[outpatient['DESY_SORT_KEY'] == mort_benes[0]]
carrier = carrier.loc[carrier['DESY_SORT_KEY'] == mort_benes[0]]

In [2]:
inpatient = claims_last_12m(inpatient, forecast_date)
outpatient = claims_last_12m(outpatient, forecast_date)
carrier = claims_last_12m(carrier, forecast_date)

In [19]:
inpatient = inpatient[inpatient_columns]
outpatient = outpatient[oupatient_columns]
carrier = carrier[carrier_columns]

In [23]:
inpatient.shape

(1, 6)

In [25]:
#saving the mort inputs for input into other script

inpatient.to_csv('inpatient_mort_claims.csv')
outpatient.to_csv('outpatient_mort_claims.csv')
carrier.to_csv('carrier_mort_claims.csv')

### hospitalization: inpatient, outpatient, carrier

In [32]:
inpatient = inpatient.loc[inpatient['DESY_SORT_KEY'] == hosp_benes[0]]
outpatient = outpatient.loc[outpatient['DESY_SORT_KEY'] == hosp_benes[0]]
carrier = carrier.loc[carrier['DESY_SORT_KEY'] == hosp_benes[0]]

In [3]:
inpatient = claims_last_12m(inpatient, forecast_date)
outpatient = claims_last_12m(outpatient, forecast_date)
carrier = claims_last_12m(carrier, forecast_date)

In [34]:
inpatient = inpatient[inpatient_columns]
outpatient = outpatient[oupatient_columns]
carrier = carrier[carrier_columns]

In [37]:
carrier.shape

(87, 4)

In [41]:
#saving the inputs for data_pull_claim_details notebook
inpatient.to_csv('inpatient_hosp_claims.csv')
outpatient.to_csv('outpatient_hosp_claims.csv')
carrier.to_csv('carrier_hosp_claims.csv')

### claim details for hha, hospice, snf, dme

In [49]:
#filtering dfs into the chosen bene 

hha = hha.loc[hha['DESY_SORT_KEY'] == chosen_bene]
hospice = hospice.loc[hospice['DESY_SORT_KEY'] == chosen_bene]
snf = snf.loc[snf['DESY_SORT_KEY'] == chosen_bene]
dme = dme.loc[dme['DESY_SORT_KEY'] == chosen_bene]

In [4]:
hha = claims_last_12m(hha, forecast_date)
hospice = claims_last_12m(hospice, forecast_date)
snf = claims_last_12m(snf, forecast_date)
dme = claims_last_12m(dme, forecast_date)

In [44]:
final_col_list = ['DESY_SORT_KEY', 'visit_type', 'start_date','primary_diagnosis_code', 'CLAIM_NO']
new_names = {'DEL_FT':'visit_type', 
             'CLM_THRU_DT': 'start_date', 
              'PRNCPAL_DGNS_CD': 'primary_diagnosis_code'}

In [51]:
hha.rename(columns=new_names, inplace=True)
hha['visit_type'] = 'HHA'
hha = hha[final_col_list]

hospice.rename(columns=new_names, inplace=True)
hospice['visit_type'] = 'Hospice'
hospice = hospice[final_col_list]

snf.rename(columns=new_names, inplace=True)
snf['visit_type'] = 'SNF'
snf = snf[final_col_list]

dme.rename(columns=new_names, inplace=True)
dme['visit_type'] = 'DME'
dme = dme[final_col_list]

In [52]:
#final df to input into other script and be concatenated

mort_claims = pd.concat([hha, hospice, snf, dme])
mort_claims.reset_index(drop=True, inplace=True)
mort_claims.head(15)

,DESY_SORT_KEY,visit_type,start_date,primary_diagnosis_code,CLAIM_NO
0,100701699,HHA,2014-10-15,V5830,3403
1,100701699,HHA,2014-10-31,25060,3404
2,100701699,HHA,2015-01-12,25080,3489
3,100701699,HHA,2015-03-13,25080,3490
4,100701699,HHA,2015-05-12,25080,3491
5,100701699,HHA,2015-07-11,70707,3492
6,100701699,HHA,2015-09-09,25080,3493
7,100701699,HHA,2015-09-26,25080,3494
8,100701699,SNF,2015-04-30,5849,2874
9,100701699,SNF,2015-05-12,5849,2875


In [46]:
#final df to input into other script and be concatenated

hosp_claims = pd.concat([hha, hospice, snf, dme])
hosp_claims.reset_index(drop=True, inplace=True)
hosp_claims.head(15)

,DESY_SORT_KEY,visit_type,start_date,primary_diagnosis_code,CLAIM_NO
0,133365053,HHA,2014-11-05,42731,84493
1,133365053,HHA,2015-03-04,5119,81082
2,133365053,SNF,2014-11-30,V5789,66868
3,133365053,SNF,2014-12-14,V5789,66869
4,133365053,DME,2014-10-24,43310,657647


In [53]:
mort_claims.to_csv('mort_claims_half.csv')
hosp_claims.to_csv('hosp_claims_half.csv')